# Train the VOLVuLuS model
Train the VOLumetric VesseL Segmentation (VOLVuLuS) model, a variant of the 3D U-Net by ([Çiçek 2016](#References)). The training set comprises **volumes of** (as opposed to **individual** images used by DECiSION) 320 by 320 pixels grayscale axial MRI images along with ground truth images highlighting blood vessels associated with each MRI image. The trained model can be applied to (unseen) MRI images to produce segmentation maps using `VOLVuLuS_test.ipynb`.

# Set seeds and import packages
Setting the seeds first is meant to achieve reproducibility, though this goal is not entirely achieved.

In [1]:
RANDOM_STATE = 42
from numpy.random import seed
seed(RANDOM_STATE)

from tensorflow import set_random_seed
set_random_seed(RANDOM_STATE)

import random
random.seed = RANDOM_STATE

# Model and training settings
import VOLVuLuS_settings as settings

# Toolkit imports
from dltoolkit.utils.generic import model_architecture_to_file, model_summary_to_file, list_images
from dltoolkit.nn.segment import UNet_3D_NN
from dltoolkit.utils.visual import plot_training_history

from thesis_common import convert_img_to_pred_3d, convert_pred_to_img_3d, create_hdf5_db_3d,\
    show_image, print_training_info, read_groundtruths, read_images, load_training_3d
from thesis_metric_loss import dice_coef_threshold, weighted_pixelwise_crossentropy_loss

# Keras imports
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, TensorBoard, ReduceLROnPlateau
from keras.optimizers import Adam, SGD

# scikit-learn imports
from sklearn.model_selection import train_test_split

# Other imports
import numpy as np
import os, cv2, time, progressbar
import matplotlib.pyplot as plt
%matplotlib inline

/Users/geoff/anaconda3/envs/ML3-DL-OPENCV/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Change how TensorFlow allocates GPU memory
Setting `gpu_options.allow_growth` to `True` means TensorFlow will allocate GPU memory as needed rather than using all available memory from the start. This enables monitoring of actual memory usage and determining how close the notebook gets to running out of memory. This has no effect on non-GPU machines.

In [2]:
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
 
# Only allow a percentage of the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified
k.tensorflow_backend.set_session(tf.Session(config=config))

# Determine training settings
The variables below determine how the model will be trained:

- `USE_VALIDATION_SET`: set to `True` to use a validation set during training, which will be the case most of the time. set to `False` to not use a validation set, e.g. during pipeline development/validation.

Contrary to DECiSION, VOLVuLuS always uses data generators due to the size of the model.

**Note** if `TRN_TRAIN_VAL_SPLIT` is set to 0, a validation set will *not* be created, even if `USE_VALIDATION_SET` is set to `True`. In fact, `USE_VALIDATION_SET` will be set to `False`.

In [3]:
USE_VALIDATION_SET = True

if not USE_VALIDATION_SET:
    # If no validation set is to be used override the split value
    settings.TRN_TRAIN_VAL_SPLIT = 0.0

# Load data
Contrary to DECiSION, data used to train VOLVuLuS is not converted to HDF5 format. Also, all data is read into memory; generators are not used. This was done to simplify the code to assist with debugging when it became apparent that training the model required huge amounts of memory. Keeping code as clean as possible ruled out issues with our code.

In [4]:
train_imgs, train_grndtr, train_grndtr_ext_conv, val_imgs, val_grndtr, val_grndtr_ext_conv, num_patients = load_training_3d(settings)

Reading images  33% |##############                             | ETA:  0:00:00

Loading training images


Reading images 100% |###########################################| Time: 0:00:00
Reading images 100% |###########################################| Time: 0:00:00
Reading images N/A% |                                          | ETA:  --:--:--

Loading training ground truths
Elapsed time: 0.11s
Loading validation images


Reading images 100% |###########################################| Time: 0:00:00
Reading images 100% |###########################################| Time: 0:00:00


Loading validation ground truths
Elapsed time: 0.03s


# Set the class distribution
Assigning a higher weight to the positive class (i.e. blood vessels) means the model will pay "more attention" to that class. This is useful in the current class imbalance scenario because the number of background (i.e. non-blood vessel) pixels far exceed the number of blood vessel pixels. Without setting a different class weight for the blood vessel class the model would simply assign the background class to all pixels to achieve a low loss.

In [5]:
class_weights = [settings.CLASS_WEIGHT_BACKGROUND, settings.CLASS_WEIGHT_BLOODVESSEL]
print("Class distribution: {}".format(class_weights))

Class distribution: [1.0, 80.0]


# Create the 3D U-Net model
Instantiate the 3D U-Net model. Use different versions of the `build_model_XXX()` function to try different variations of the model. **Warning**: Changing the model and/or its parameters will change the name of the file the trained model will be saved to. Make sure to update the `VOLVuLuS_test_ipynb` notebook accordingly to ensure it uses the correct saved model.

In [6]:
unet = UNet_3D_NN(img_height=settings.IMG_HEIGHT,
                  img_width=settings.IMG_WIDTH,
                  num_slices=settings.SLICE_END - settings.SLICE_START,
                  img_channels=settings.IMG_CHANNELS,
                  num_classes=settings.NUM_CLASSES)

model = unet.build_model_alt(num_layers=settings.MDL_LAYERS,
                             n_base_filters=settings.MDL_BASE_FLTRS,
                             deconvolution=settings.MDL_DECON,
                             use_bn=settings.MDL_BN)

# Create paths
This cell just creates a few paths used later to save training output (e.g. the model architecture, training results and so on).

In [7]:
prefix = "VOLVuLuS_" + unet.title + "_W"+ str(settings.CLASS_WEIGHT_BLOODVESSEL) + "_" + settings.TRN_LOSS + "_BS" + "{:03}".format(settings.TRN_BATCH_SIZE)

model_path = os.path.join(settings.MODEL_PATH, prefix + ".model")
summ_path = os.path.join(settings.OUTPUT_PATH, prefix + "_model_summary.txt")
csv_path = os.path.join(settings.OUTPUT_PATH, prefix + "_training.csv")

# Save/print model architecture information
Save the model's architecture to a file, print it in the cell below and save a diagram to disk.

In [8]:
model.summary()
model_summary_to_file(model, summ_path)
model_architecture_to_file(unet.model, os.path.join(settings.OUTPUT_PATH, prefix))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 32, 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 256, 256, 32, 448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 32, 64          conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 256, 256, 32, 13856       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

# Compile the model
Set the loss function, optimiser and metric and compile the model.

In [9]:
# Set the optimiser, loss function and metrics
if settings.TRN_LOSS == "ADAM":
    opt = Adam(lr=settings.TRN_LEARNING_RATE, amsgrad=settings.TRN_AMS_GRAD)
else:
    opt = SGD(lr=settings.TRN_LEARNING_RATE)

# Softmax:
metrics = [dice_coef_threshold(settings.TRN_PRED_THRESHOLD)]
loss = weighted_pixelwise_crossentropy_loss(class_weights)

# Sigmoid:
# metrics = [dice_coef]
# loss = "binary_crossentropy"

# Compile
model.compile(optimizer=opt, loss=loss, metrics=metrics)

# Prepare callbacks
Prepare callbacks used during training:

- TensorBoard: basic TensorBoard visualizations (not always used)
- EarlyStopping: Stop training when a monitored quantity has stopped improving
- CSVLogger: streams epoch results to a csv file
- ModelCheckpoint: save the model after every epoch
- ReduceLROPlateau: reduce the learning when progress halts

In [ ]:
if USE_VALIDATION_SET:
    loss_str = "val_loss"
else:
    loss_str = "loss"

# tb_callb = TensorBoard(log_dir=settings.OUTPUT_PATH + unet.title,
#                        write_graph=True,
#                        batch_size=settings.TRN_BATCH_SIZE)

cvs_callb = CSVLogger(csv_path, append=False)


red_callb = ReduceLROnPlateau(monitor=loss_str,
                          factor=settings.TRN_PLAT_FACTOR,
                          patience=settings.TRN_PLAT_PATIENCE,
                          verbose=1,
                          mode="min")

mc_callb = ModelCheckpoint(model_path,
                           monitor=loss_str,
                           mode="min",
                           save_best_only=True,
                           save_weights_only=True,
                           verbose=1)

es_callb = EarlyStopping(monitor=loss_str,
                         min_delta=0,
                         patience=settings.TRN_EARLY_PATIENCE,
                         verbose=0,
                         mode="auto")

callbacks = [mc_callb, es_callb, cvs_callb, red_callb]

## Train the model
Execute the training process. All data is loaded into memory.

In [ ]:
start_time = time.time()

if USE_VALIDATION_SET:
    print("Training with a validation set, using all data in memory.")
    print_training_info(unet, model_path, train_imgs.shape, val_imgs.shape,
                        settings, class_weights, num_patients, opt, loss)

    # Fit the model using generators and a validation set
    hist = model.fit(train_imgs, train_grndtr_ext_conv,
                     epochs=settings.TRN_NUM_EPOCH,
                     batch_size=settings.TRN_BATCH_SIZE,
                     verbose=2,
                     shuffle=True,
                     validation_data=(val_imgs, val_grndtr_ext_conv),
                     callbacks=callbacks
                    )

else:
    print("Training without a validation set, using all data in memory.")
    print_training_info(unet, model_path, train_imgs.shape, None,
                        settings, class_weights, num_patients, opt, loss)

    # Fit the model using a training set only
    start_time = time.time()
    hist = model.fit(train_imgs, train_grndtr_ext_conv,
                     epochs=settings.TRN_NUM_EPOCH,
                     batch_size=settings.TRN_BATCH_SIZE,
                     verbose=2,
                     shuffle=True,
                     callbacks=callbacks
                    )

print("\n\nElapsed training time: {:.2f} min.".format(int((time.time() - start_time))/60))

# Perform pipeline test
Use the trained model on one sample in the training data set. This is just to perform pipeline testing during development.

In [ ]:
PATIENT_ID = 0
IX = 0

# For pipeline testing only
predictions = model.predict(train_imgs, batch_size=settings.TRN_BATCH_SIZE, verbose=2)

# Transpose images and ground truths to the correct oder
train_imgs_tr = np.transpose(train_imgs, axes=(0, 3, 1, 2, 4))
train_grndtr_tr = np.transpose(train_grndtr, axes=(0, 3, 1, 2, 4))

# predictions = predictions
predictions_imgs = convert_pred_to_img_3d(predictions,
                                       threshold=settings.TRN_PRED_THRESHOLD,
                                       verbose=settings.VERBOSE)

show_image(np.squeeze(train_imgs_tr[0, 1]), 'PRED TRAIN org image')
show_image(np.squeeze(train_grndtr_tr[0, 1]), 'PRED TRAIN org ground truth')
show_image(np.squeeze(predictions_imgs[0, 1]), 'PRED TRAIN predicted mask')

print("  original {} dtype {}".format(np.max(train_imgs_tr[PATIENT_ID,IX]),
                                      train_imgs_tr[PATIENT_ID,IX].dtype))
print("  gr truth {} dtype {}".format(np.max(train_grndtr_tr[PATIENT_ID,IX]),
                                      train_grndtr_tr[PATIENT_ID,IX].dtype))
print("prediction {} dtype {}".format(np.max(predictions_imgs[PATIENT_ID,IX]),
                                      predictions_imgs[PATIENT_ID,IX].dtype))

# Plot/save the training results
Show a plot of the training loss and Dice coefficient by epoch and save it to disk.

In [ ]:
plot_training_history(hist,
                      show=True,
                      save_path=os.path.join(settings.OUTPUT_PATH, prefix),
                      time_stamp=True,
                      metric="dice_coef_t")

# Training complete
The trained model is now ready to be applied to test MRI images using `VOLuLuS_test.ipynb`.

# References

*[Cicek]*: Özgün Çiçek, Ahmed Abdulkadir, Soeren S Lienkamp, Thomas Brox, and Olaf Ronneberger. 3d u-net: learning dense volumetric segmentation from sparse annotation. In *International Conference on Medical Image Computing and Computer- Assisted Intervention*, pages 424–432. Springer, 2016.